In [1]:
from transformers import AutoModelForCausalLM
from pet import get_pet_config,get_pet_model, get_pet_model_state_dict, set_pet_model_state_dict, PrefixTuningConfig, TaskType, pet_model_load_and_dispatch, bloom_model_postprocess_past_key_value, PETType
import torch
from datasets import load_dataset
import os
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator,get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
pet_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, 
                                num_virtual_tokens=30, 
                                postprocess_past_key_value_function=bloom_model_postprocess_past_key_value)

dataset_name = "twitter_complaints"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{pet_config.pet_type}_{pet_config.task_type}_v1.pt".replace("/", "_")
text_column = "Tweet text"
label_column = "text_label"
max_length=64
lr = 3e-2
num_epochs = 50
batch_size=8



In [2]:
from datasets import load_dataset

dataset = load_dataset("ought/raft", dataset_name)

classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
print(classes)
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
    
)
print(dataset)
dataset["train"][0]

Found cached dataset raft (/home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-05388978db6af01d.arrow
Loading cached processed dataset at /home/sourab/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-e3fade69c4ae889a.arrow


['Unlabeled', 'complaint', 'no complaint']
DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [3]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        #print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids 
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    #print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(max_length-len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0]*(max_length-len(sample_input_ids)) + model_inputs["attention_mask"][i]
        labels["input_ids"][i] =  [-100]*(max_length-len(sample_input_ids)) + label_input_ids 
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length]) 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs



processed_datasets = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
        )

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]


train_dataloader = DataLoader(
        train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
    )
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)




    

3


Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

In [4]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    #print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(max_length-len(sample_input_ids)) + sample_input_ids
        model_inputs["attention_mask"][i] = [0]*(max_length-len(sample_input_ids)) + model_inputs["attention_mask"][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs

test_dataset = dataset["test"].map(
            test_preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=dataset["train"].column_names,
            load_from_cache_file=False,
            desc="Running tokenizer on dataset",
        )

test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
next(iter(test_dataloader))

Running tokenizer on dataset:   0%|          | 0/4 [00:00<?, ?ba/s]

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
          227985,   5484,    915,   2566,  74757,  64626,  12384,  44639,    613,
           52282,   2670,  79920,   3344,   1002,    368,  17646,  14472,   8348,
             664,    718,      4,  19036,     17,  31849,     17,   6312,     76,
              44,  62470,     56,     91,     50,  14839,     21,  77658,    915,
             210],
         [     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3, 227985,   5484,    915,    405, 187059,
            2256,    664,   2550,  18833,  18607, 162467,      4, 

In [5]:
next(iter(train_dataloader))

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3, 227985,   5484,    915,   2566,
           15157,   4867,  14731, 165189,   2021,    769,  11528,   7220,  35025,
             530,  27937, 149533,   1965,  43435, 163255,   1141,   3611,     17,
           30655,    632,   1119,     17,  77658,    915,    210,  16449,   5952,
               3],
         [     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3, 227985,   5484,    915,   5673,    473,
           11229,   2213,   2670,  35307,  28629,    461,   2566,   2765,   1531,
            3470,  47134,  10144,   2765,   1531,    427,   2909,  17918,   6782,
           27268,   4390,   1517,     17,   3904,    632,    267, 

In [6]:
len(test_dataloader)

425

In [7]:
next(iter(test_dataloader))

{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
          227985,   5484,    915,   2566,  74757,  64626,  12384,  44639,    613,
           52282,   2670,  79920,   3344,   1002,    368,  17646,  14472,   8348,
             664,    718,      4,  19036,     17,  31849,     17,   6312,     76,
              44,  62470,     56,     91,     50,  14839,     21,  77658,    915,
             210],
         [     3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3,      3,      3,      3,      3,      3,
               3,      3,      3,      3, 227985,   5484,    915,    405, 187059,
            2256,    664,   2550,  18833,  18607, 162467,      4, 

In [8]:

# creating model
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_pet_model(model, pet_config)
model.print_trainable_parameters()



trainable params: 1474560 || all params: 560689152 || trainable%: 0.26299064191632515


In [9]:
model.print_trainable_parameters()

trainable params: 1474560 || all params: 560689152 || trainable%: 0.26299064191632515


In [10]:
model

PETModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0): BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
        (1): Bl

In [11]:
model.pet_config

PrefixTuningConfig(pet_type=<PETType.PREFIX_TUNING: 'PREFIX_TUNING'>, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=30, token_dim=1024, num_transformer_submodules=1, num_attention_heads=16, num_layers=24, encoder_hidden_size=1024, prefix_projection=False, postprocess_past_key_value_function=<function bloom_model_postprocess_past_key_value at 0x7f6fddc456c0>)

In [12]:
# model
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [13]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
#         print(batch)
#         print(batch["input_ids"].shape)
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True))

    eval_epoch_loss = eval_loss/len(train_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss/len(eval_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")


100%|█████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.43it/s]


epoch=0: train_ppl=tensor(4.1394e+09, device='cuda:0') train_epoch_loss=tensor(22.1438, device='cuda:0') eval_ppl=tensor(1402.4835, device='cuda:0') eval_epoch_loss=tensor(7.2460, device='cuda:0')


100%|█████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.49it/s]


epoch=1: train_ppl=tensor(246.8958, device='cuda:0') train_epoch_loss=tensor(5.5090, device='cuda:0') eval_ppl=tensor(62.6347, device='cuda:0') eval_epoch_loss=tensor(4.1373, device='cuda:0')


100%|█████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.48it/s]


...

epoch=47: train_ppl=tensor(1.0021, device='cuda:0') train_epoch_loss=tensor(0.0021, device='cuda:0') eval_ppl=tensor(1.0021, device='cuda:0') eval_epoch_loss=tensor(0.0021, device='cuda:0')


100%|█████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.43it/s]


epoch=48: train_ppl=tensor(1.0020, device='cuda:0') train_epoch_loss=tensor(0.0020, device='cuda:0') eval_ppl=tensor(1.0021, device='cuda:0') eval_epoch_loss=tensor(0.0021, device='cuda:0')


100%|█████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 22.42it/s]

epoch=49: train_ppl=tensor(1.0020, device='cuda:0') train_epoch_loss=tensor(0.0020, device='cuda:0') eval_ppl=tensor(1.0021, device='cuda:0') eval_epoch_loss=tensor(0.0021, device='cuda:0')


In [14]:
model.eval()
i = 12
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")
print(dataset["test"][i]["Tweet text"])
print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))
    

@VW @QuirkCars some unauthorized work done on my engine now throwing a check engine light about a week later. Very upset.
{'input_ids': tensor([[227985,   5484,    915,   2566,     57,     58,   2566,   5232, 132511,
             38,   4599,   3331,   1035, 192352,   2909,  11541,    664,   2670,
          22218,   5840, 108218,    267,   7010,  22218,  12490,   3638,    267,
          14319,  10494,     17,  93269, 123055,     17,  77658,    915,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[227985,   5484,    915,   2566,     57,     58,   2566,   5232, 132511,
             38,   4599,   3331,   1035, 192352,   2909,  11541,    664,   2670,
          22218,   5840, 108218,    267,   7010,  22218,  12490,   3638,    267,
          14319,  10494,     17,  93269, 123055,     17,  77658,    915,    210,
          16449,   5952,      3,      3,      3,      3,      3,  

In [ ]:
model.eval()
eval_loss = 0
eval_preds = []
for step, batch in enumerate(tqdm(test_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items() if k!="labels"}
    with torch.no_grad():
        outputs = model.generate(**batch, max_new_tokens=10)
    eval_preds.extend(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))



In [15]:
# saving model
state_dict = get_pet_model_state_dict(model)
torch.save(state_dict, checkpoint_name)
print(state_dict)

{'prompt_embeddings': tensor([[ 0.7356, -1.0849, -0.4560,  ...,  1.0242,  0.3908, -0.8000],
        [-1.5587, -0.3595,  0.1289,  ...,  0.5427,  1.0976,  1.7641],
        [-0.2113, -1.4675, -0.5976,  ...,  0.1691, -0.5843, -0.2658],
        ...,
        [ 2.1275,  0.7253,  0.0323,  ..., -1.2285, -0.5614,  0.0370],
        [-0.2258, -1.5149,  0.0685,  ..., -1.4476, -0.1348, -0.6910],
        [ 0.9089,  0.3947, -1.5271,  ...,  1.9079,  0.6473,  0.7306]])}


In [16]:
!du -h $checkpoint_name

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
5,7M	twitter_complaints_bigscience_bloomz-560m_PREFIX_TUNING_CAUSAL_LM_v1.pt


In [8]:
max_memory={0: "1GIB", 1: "1GIB", 2: "2GIB", 3: "2GIB", "cpu":"30GB"}

pet_config.inference_mode = True
print(pet_config)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map="auto", max_memory=max_memory)
model = pet_model_load_and_dispatch(model, torch.load(checkpoint_name), pet_config, max_memory)



PrefixTuningConfig(pet_type=<PETType.PREFIX_TUNING: 'PREFIX_TUNING'>, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=True, num_virtual_tokens=30, token_dim=None, num_transformer_submodules=1, num_attention_heads=None, num_layers=None, encoder_hidden_size=None, prefix_projection=False, postprocess_past_key_value_function=<function bloom_model_postprocess_past_key_value at 0x7f913d791630>)
trainable params: 1474560 || all params: 560689152 || trainable%: 0.26299064191632515


In [9]:
model

PETModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(250880, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0): BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
        (1): Bl

In [10]:
model.hf_device_map

{'base_model.transformer.word_embeddings': 3,
 'word_embeddings': 3,
 'base_model.transformer.word_embeddings_layernorm': 3,
 'base_model.transformer.h.0': 3,
 'base_model.transformer.h.1': 3,
 'base_model.transformer.h.2': 3,
 'base_model.transformer.h.3': 3,
 'base_model.transformer.h.4': 3,
 'base_model.transformer.h.5': 3,
 'base_model.transformer.h.6': 3,
 'base_model.transformer.h.7': 3,
 'base_model.transformer.h.8': 3,
 'base_model.transformer.h.9': 3,
 'base_model.transformer.h.10': 3,
 'base_model.transformer.h.11': 3,
 'base_model.transformer.h.12': 3,
 'base_model.transformer.h.13': 3,
 'base_model.transformer.h.14': 3,
 'base_model.transformer.h.15': 3,
 'base_model.transformer.h.16': 3,
 'base_model.transformer.h.17': 3,
 'base_model.transformer.h.18': 3,
 'base_model.transformer.h.19': 3,
 'base_model.transformer.h.20': 3,
 'base_model.transformer.h.21': 3,
 'base_model.transformer.h.22': 'cpu',
 'base_model.transformer.h.23': 'cpu',
 'base_model.transformer.ln_f': 'cpu'

In [11]:
model.eval()
i = 12
inputs = tokenizer(f'{text_column} : {dataset["test"][i]["Tweet text"]} Label : ', return_tensors="pt")
print(dataset["test"][i]["Tweet text"])
print(inputs)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))


@VW @QuirkCars some unauthorized work done on my engine now throwing a check engine light about a week later. Very upset.
{'input_ids': tensor([[227985,   5484,    915,   2566,     57,     58,   2566,   5232, 132511,
             38,   4599,   3331,   1035, 192352,   2909,  11541,    664,   2670,
          22218,   5840, 108218,    267,   7010,  22218,  12490,   3638,    267,
          14319,  10494,     17,  93269, 123055,     17,  77658,    915,    210]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[227985,   5484,    915,   2566,     57,     58,   2566,   5232, 132511,
             38,   4599,   3331,   1035, 192352,   2909,  11541,    664,   2670,
          22218,   5840, 108218,    267,   7010,  22218,  12490,   3638,    267,
          14319,  10494,     17,  93269, 123055,     17,  77658,    915,    210,
          16449,   5952,      3,      3,      3,      3,      3,  